<td>   <a target="_blank" href="https://labelbox.com" ><img src="https://labelbox.com/blog/content/images/2021/02/logo-v4.svg" width=256/></a></td>


<td>
<a href="https://colab.research.google.com/github/Labelbox/labelbox-python/blob/develop/examples/basics/projects.ipynb" target="_blank"><img
src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"></a>
</td>

<td>
<a href="https://github.com/Labelbox/labelbox-python/tree/develop/examples/basics/projects.ipynb" target="_blank"><img
src="https://img.shields.io/badge/GitHub-100000?logo=github&logoColor=white" alt="GitHub"></a>
</td>

# Projects

* A project can be thought of as a specific labeling task on a set of labels
* That set of labels is defined by the data rows attached to the project
* Each project has an ontology which defines the types of annotations supported during the labeling process
**Note that there is a lot of advanced usage that is not covered in this notebook. See examples/project_configuration/project_setup.ipynb for those functions**
* Also note that deprecated functions are not explained here.

In [ ]:
%pip install -q "labelbox[data]"

In [ ]:
import labelbox as lb
import labelbox.types as lb_types
from labelbox.schema.conflict_resolution_strategy import (
    ConflictResolutionStrategy,)
import uuid

# API Key and Client
Provide a valid api key below in order to properly connect to the Labelbox Client.

In [ ]:
# Add your API key
API_KEY = ""
# To get your API key go to: Workspace settings -> API -> Create API Key
client = lb.Client(api_key=API_KEY)

### Create a project


In [ ]:
# Creates an empty project
project = client.create_project(
    name="my-test-project",
    description="a description",
    media_type=lb.MediaType.Image,
)

### Create a dataset with data rows

In [ ]:
dataset = client.create_dataset(name="project-demo-dataset")
global_keys = []
uploads = []
# Generate data rows
for i in range(1, 9):
    gb_key = "TEST-ID-%id" % uuid.uuid1()
    uploads.append({
        "row_data":
            f"https://storage.googleapis.com/labelbox-datasets/People_Clothing_Segmentation/jpeg_images/IMAGES/img_000{i}.jpeg",
        "global_key":
            gb_key,
    })
    global_keys.append(gb_key)

task = dataset.create_data_rows(uploads)
task.wait_till_done()
print("ERRORS: ", task.errors)
print("RESULT URL: ", task.result_url)

### Add data rows to a project 


In [ ]:
project.create_batch(
    "project-demo",  # each batch in a project must have a unique name
    global_keys=
    global_keys,  # paginated collection of data row objects, list of data row ids or global keys
    priority=1,  # priority between 1(highest) - 5(lowest)
)

### Create tags and assign them to a project
In this section, we are creating a tag in the ontology and associating it with a project. Then we are listing the tags attached to a project.


#### Create a tag

In [ ]:
# Get the organization
organization = client.get_organization()

tag = organization.create_resource_tag({
    "text": "new-tag-name",
    "color": "4ed2f9"
})

#### Assign the tag to a project

In [ ]:
tags = project.update_project_resource_tags([tag.uid])

#### Get project tags

In [ ]:
tags = project.get_resource_tags()

### Attach ontology and label data rows

In this section, we are creating an ontology to attach to a project and creating labels to import as ground truths. We need this setup to demonstrate other methods later in the demo. For more information, please reference our [Ontology](https://docs.labelbox.com/reference/ontology) and [Import Image Annotation](https://docs.labelbox.com/reference/import-image-annotations) development guides.

#### Create your ontology

In [ ]:
# Create normalized json with a radio classification
ontology_builder = lb.OntologyBuilder(
    classifications=[  # List of Classification objects
        lb.Classification(
            class_type=lb.Classification.Type.RADIO,
            name="radio_question",
            options=[
                lb.Option(value="first_radio_answer"),
                lb.Option(value="second_radio_answer"),
            ],
        ),
    ])
# Creating an ontology
ontology = client.create_ontology("test-ontology", ontology_builder.asdict())

#### Attach ontology to project

In [ ]:
project.setup_editor(ontology)

### Create labels and upload them to project as ground truths

In [ ]:
# Create labels
labels = []
for global_key in global_keys:
    labels.append(
        lb_types.Label(
            data={"global_key": global_key},
            annotations=[
                # Create radio classification annotation for labels
                lb_types.ClassificationAnnotation(
                    name="radio_question",
                    value=lb_types.Radio(answer=lb_types.ClassificationAnswer(
                        name="second_radio_answer")),
                )
            ],
        ))

# Upload labels for the data rows in project
upload_job = lb.LabelImport.create_from_objects(
    client=client,
    project_id=project.uid,
    name="label_import_job" + str(uuid.uuid4()),
    labels=labels,
)

upload_job.wait_until_done()

print(f"Errors: {upload_job.errors}")

### Move data rows in project to different task queues

In [ ]:
# Get list of task queues for project
task_queues = project.task_queues()

for task_queue in task_queues:
    print(task_queue)

In [ ]:
project.move_data_rows_to_task_queue(
    data_row_ids=lb.GlobalKeys(global_keys),  # Provide a list of global keys
    task_queue_id=task_queues[2].
    uid,  # Passing None moves data rows to "Done" task queue
)

### Fetch project configuration

In [ ]:
# Note the project is not fully setup many of the fields will be empty.
print("Project is not setup yet:", project.setup_complete is None)
print("Project name:", project.name)
print("Project description:", project.description)
print("Media Type:", project.media_type)
batches = [b for b in project.batches()]
print("Project Batches", batches)
print("Ontology:", project.ontology())

### Return number of labeled data rows

In [ ]:
print("Number of labels:", project.get_label_count())

### Duplicate a project
Please see the section [Duplicate a project](https://docs.labelbox.com/docs/create-a-project#duplicate-a-project) to have the scope of the method.

In [ ]:
destination_project_id = project.clone()

destination_project = client.get_project(destination_project_id)

### Copy labels and data rows from one project to a different project
In the below steps we will be copying data rows with their corresponding labels from one project to a different project with a similar ontology. First, we must set up a new project with a ontology that matches the tooling of our source project ontology.

In [ ]:
# Create an empty destination project
destination_project = client.create_project(
    name="destination-test-project",
    description="a description",
    media_type=lb.MediaType.Image,
)

# Create ontology and attach to destination project
destination_ontology_builder = lb.OntologyBuilder(
    classifications=[  # List of Classification objects
        lb.Classification(
            class_type=lb.Classification.Type.RADIO,
            name="destination_radio_question",
            options=[
                lb.Option(value="destination_first_radio_answer"),
                lb.Option(value="destination_second_radio_answer"),
            ],
        ),
    ])

destination_ontology = client.create_ontology("dest-test-ontology",
                                              ontology_builder.asdict())

destination_project.setup_editor(destination_ontology)

#### Copy data rows and labels
To copy our data rows and labels to our project from a source project we will be using the `send_to_annotate_from_catalog` method with our Labelbox client.

##### Parameters

When you send data rows with labels to our destination project, you may choose to include or exclude certain parameters, at a minimum a `source_project_id` will need to be provided:

* `source_project_id`
    - The id of the project were our data rows with labels will originate.
* `annotation_ontology_mapping`
    - A dictionary containing the mapping of the source project's ontology feature schema ids to the destination project's ontology feature schema ids. If left empty only the data rows will be sent to our destination project with no labels.
* `exclude_data_rows_in_project`
    - Excludes data rows that are already in the project. 
* `override_existing_annotations_rule` 
    - The strategy defining how to handle conflicts in classifications between the data rows that already exist in the project and incoming labels from the source project. 
        * Defaults to ConflictResolutionStrategy.KeepExisting
        * Options include:
            * ConflictResolutionStrategy.KeepExisting
            * ConflictResolutionStrategy.OverrideWithPredictions
            * ConflictResolutionStrategy.OverrideWithAnnotations
* `param batch_priority`
    - The priority of the batch.

In [ ]:
# Get ontology dictionary to obtain featureSchemaIds
source_ontology_normalized = ontology.normalized
destination_ontology_normalized = destination_ontology.normalized

ANNOTATION_ONTOLOGY_MAPPING = {
    source_ontology_normalized["classifications"][0]["featureSchemaId"]:
        destination_ontology_normalized["classifications"][0]
        ["featureSchemaId"],  # Classification featureSchemaID
    source_ontology_normalized["classifications"][0]["options"][0]["featureSchemaId"]:
        destination_ontology_normalized["classifications"][0]["options"][0]
        ["featureSchemaId"],  # Different Classification Answer featureSchemaIDs
    source_ontology_normalized["classifications"][0]["options"][1]["featureSchemaId"]:
        destination_ontology_normalized["classifications"][0]["options"][1]
        ["featureSchemaId"],
}

In [ ]:
send_to_annotate_params = {
    "source_project_id":
        project.uid,
    "annotations_ontology_mapping":
        ANNOTATION_ONTOLOGY_MAPPING,
    "exclude_data_rows_in_project":
        False,
    "override_existing_annotations_rule":
        ConflictResolutionStrategy.OverrideWithPredictions,
    "batch_priority":
        5,
}

# Get task id to workflow you want to send data rows. If sent to initial labeling queue, labels will be pre-labels.
queue_id = [
    queue.uid
    for queue in destination_project.task_queues()
    if queue.queue_type == "MANUAL_REVIEW_QUEUE"
][0]

task = client.send_to_annotate_from_catalog(
    destination_project_id=destination_project.uid,
    task_queue_id=
    queue_id,  # ID of workflow task, set ID to None if you want to send data rows with labels to the Done queue.
    batch_name="Prediction Import Demo Batch",
    data_rows=lb.GlobalKeys(
        global_keys  # Provide a list of global keys from source project
    ),
    params=send_to_annotate_params,
)

task.wait_till_done()

print(f"Errors: {task.errors}")

### Clean Up

In [ ]:
# project.delete()
# destination_project.delete()
# dataset.delete()
# client.delete_unused_ontology(destination_ontology.uid)
# client.delete_unused_ontology(ontology.uid)